In [1]:
import data_prepare
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
print(torch.cuda.is_available())

True


In [2]:
path = 'E:\PersonalFiles\杂项\Dissertation\Data/diabetes_data.csv'
x_train, x_test, y_train, y_test = data_prepare.split_data(path)

In [3]:
class MyData(Dataset):
    def __init__(self, x, y):
        self.x = np.array(x)
        self.y = np.array(y)
        
        self.x = torch.tensor(self.x, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.y)
    
train_set = MyData(x_train, y_train)
test_set  = MyData(x_test,  y_test)

batch_size = 60
# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
# test_loader  = DataLoader(test_set, batch_size=batch_size, drop_last=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.seq(x)
        x = torch.squeeze(x, 1)
        return x
    
net = Net()
lr = 1e-2
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss_fn = nn.BCELoss()

In [5]:
epochs = 20
kf = KFold(n_splits=5)

for i, (train_idx, test_idx) in enumerate(kf.split(train_set)):
    train_data = [train_set[i] for i in train_idx]
    test_data = [train_set[i] for i in test_idx]
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader  = DataLoader(test_data, batch_size=batch_size, drop_last=True)

    print(f'index of KFold: {i}')
    for e in range(epochs):
        true_num = 0
        total_num = 0
        training_loss = 0.
        for idx, (data, target) in enumerate(train_loader):
            output = net(data)
            # print(output.shape, data.shape, target.shape)
            loss = loss_fn(output, target)
            training_loss += loss

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_num += len(target)

            true_num += ((output > 0.5) == (target > 0.5)).sum().item()

        # print(true_num)
        
        print(f'epoch: {e}, loss: {training_loss:.2f},\
              accuracy on training set: {true_num/total_num*100 :.2f}')

index of KFold: 0
epoch: 0, loss: 2.68,              accuracy on training set: 65.83
epoch: 1, loss: 1.87,              accuracy on training set: 82.92
epoch: 2, loss: 0.97,              accuracy on training set: 91.25
epoch: 3, loss: 0.62,              accuracy on training set: 92.50
epoch: 4, loss: 0.77,              accuracy on training set: 93.33
epoch: 5, loss: 0.74,              accuracy on training set: 93.33
epoch: 6, loss: 0.53,              accuracy on training set: 94.17
epoch: 7, loss: 0.47,              accuracy on training set: 95.83
epoch: 8, loss: 0.30,              accuracy on training set: 98.33
epoch: 9, loss: 0.39,              accuracy on training set: 96.67
epoch: 10, loss: 0.27,              accuracy on training set: 97.92
epoch: 11, loss: 0.29,              accuracy on training set: 97.50
epoch: 12, loss: 0.22,              accuracy on training set: 99.17
epoch: 13, loss: 0.24,              accuracy on training set: 98.33
epoch: 14, loss: 0.22,              accu

In [6]:
for idx, (data, target) in enumerate(test_loader):
    output = net(data)
    # print(output.shape, data.shape, target.shape)
    total_num += len(target)

    true_num += ((output > 0.5) == (target > 0.5)).sum().item()

# print(true_num)

print(f'accuracy on testing set: {true_num/total_num*100 :.2f}')

accuracy on testing set: 99.33


In [9]:
x = 100 - 93.19
y = 100 - 99.50
(x-y)/x

0.9265785609397944

In [13]:
( 89.40+ 94.24+ 92.66)/3

92.09999999999998